In [36]:
import netCDF4 as nc
import pandas as pd
import numpy as np
 
# Variable to extract
ssp = 'ssp585'  # ssp126 ssp245 ssp585
scenario = ssp[3:]
# Work with one at a time
var = 'thetao' #change here for --> o2 chl thetao ph so
variable_name = 'THETAO' #change here for --> Dissolved Oxygen Chlorophyll Thetao pH Salinity

# Loc change for each variable
variable_location = 'surface_depth_5.0'  # 'surface_depth_5.0' 'surface_depth_25.0' 'bottom'

# File Path for NetCDF file -> This would be your filepath where the FUMA dataset is stored.
filepath = f'H:/M.Sc Thesis - Data, Methodology, Results/NetCDF Files/FutureMares North Sea Package/{ssp}/ensemble/{var}/{var}_ensemble_sd+ba_{variable_location}_perc_{ssp}.nc'   #change here

# List of locations only in North Sea with their coordinates
locations = [
    {'Location Name': 'FINO3', 'Nlat': 55.166668, 'Nlon': 7.1666718},
    {'Location Name': 'BORSSELE', 'Nlat': 51.75, 'Nlon': 3.0},
    {'Location Name': 'BELWIND', 'Nlat': 51.666668, 'Nlon': 2.8333282}
]

# Open the NetCDF file and read data
with nc.Dataset(filepath, mode='r') as dataset:
    latitudes = dataset.variables['lat'][:]
    longitudes = dataset.variables['lon'][:]
    times = dataset.variables['time'][:]
    variable = dataset.variables[var]                    #change here
    time_units = dataset.variables['time'].units
    dates = nc.num2date(times, units=time_units)
    
    # Convert the dates to strings just once, as they are the same for all locations
    formatted_dates = [date.strftime('%d %b %Y') for date in dates]
    
    # Extract and process data for each location
    for location in locations:
        closest_lat_index = np.argmin(np.abs(latitudes - location['Nlat']))
        closest_lon_index = np.argmin(np.abs(longitudes - location['Nlon']))
        # print(closest_lat_index, closest_lon_index)
        # DataFrames to hold temperature data for each percentile
        percentile_indices = {'P2.5': 0, 'P50': 1, 'P97.5': 2}  # Adjust indices as necessary
        temp_data = {
            'Date': formatted_dates
        }
        for label, idx in percentile_indices.items():
            specific_values = variable[:, closest_lat_index, closest_lon_index, idx]          #change here
            temp_data[f'{variable_name} ({label})'] = specific_values                  #change here

        # Create a DataFrame for the location
        location_df = pd.DataFrame(temp_data)

        # # Save the DataFrame to a CSV file named after the location
        # location_df.to_csv(f"FUMA{location['Location Name']}_{var}_{scenario}.csv", index=False)   #change here for your desired format. I am extracting all surface 25.0 values so I didn't specify {variable_location}
        # #But this script will work for all FUMA nc files provided you have the locations and stuff noted.

print("Data processing for all locations complete and files have been saved.")

Data processing for all locations complete and files have been saved.
